## Reading Flow Cytometry data into R

v1.0 (2021-03-29)  
Lucas Graybuck  

### Purpose

In this notebook, we'll see how to read gating counts and flow cytometry .fcs files into R for analysis.

This notebook was generated using the `R` language, running in the Jupyter Notebook environment on a HISE IDE instance. See the end of the document for the [Session Info](#Session-Info) for additional software version details.

<a name = "contents"></a>

### Contents

- [Importing packages](#Importing-packages)
- [Reading gating counts](#Reading-gating-counts)
- [Reading .fcs files](#Reading-.fcs-files)
- [Session Info](#Session-Info)

Ingest fcs w/flowkit

basic demo w/flowkit

### Importing packages

The [`flowkit` package](https://flowkit.readthedocs.io/en/latest/) for Python provides basic read/write functions and some analysis for flow cytometry data.

It can be installed from a Terminal using:
```
pip install cmake
pip install flowkit
```

Once installed, we can load flowkit, along with `pandas` and `numpy` for reading and manipulating .csv files containing automated gating counts.

In [1]:
import flowkit
import pandas
import numpy
import os
import re

We'll also set up some `bokeh` settings - this is an interactive plotting package that's used by `flowkit` to display plots:

In [2]:
import bokeh
from bokeh.plotting import show

bokeh.io.output_notebook()
%matplotlib inline

Loading BokehJS ...

### Reading gating counts

In Python, we can't read counts directly in from HISE (for now). So, we need to rely on files downloaded by the R `hise` package.

For this demonstration, we'll use the counts files we obtained in the notebook `01-R Retrieving data from HISE.ipynb`:

In [3]:
cache_info = pandas.read_csv('cache_info.csv')
cache_info.columns

Index(['Unnamed: 0', 'filePath', 'lastUpdated', 'sample.id',
       'sample.bridgingControl', 'sample.sampleKitGuid', 'sample.visitName',
       'sample.drawDate', 'sample.daysSinceFirstVisit', 'file.id', 'file.name',
       'file.batchID', 'file.panel', 'file.pool', 'file.fileType',
       'subject.id', 'subject.biologicalSex', 'subject.birthYear',
       'subject.ethnicity', 'subject.partnerCode', 'subject.race',
       'subject.subjectGuid', 'cohort.cohortGuid'],
      dtype='object')

In [4]:
cache_info['file.fileType'].value_counts()

FlowCytometry-supervised-stats    8
scRNA-seq-labeled                 5
FlowCytometry                     1
atac-assembly-archr-arrow         1
Name: file.fileType, dtype: int64

In [5]:
counts_info = cache_info[cache_info['file.fileType'] == 'FlowCytometry-supervised-stats']
counts_info.shape

(8, 23)

In [6]:
counts_files = counts_info['filePath']

Now we can read these in using pandas:

In [7]:
counts_list = []
for file in counts_files:
    counts_list.append(pandas.read_csv(file))

In [8]:
counts_list[0].head()

,name,Population,Parent,Count,ParentCount
0,B045_PS1_PB00546-01_QC.fcs,/Cells,root,290412,313336
1,B045_PS1_PB00546-01_QC.fcs,/Cells/Singlets-H,/Cells,275575,290412
2,B045_PS1_PB00546-01_QC.fcs,/Cells/Singlets-H/Singlets-W,/Cells/Singlets-H,271763,275575
3,B045_PS1_PB00546-01_QC.fcs,/Cells/Singlets-H/Singlets-W/Cleanup,/Cells/Singlets-H/Singlets-W,267721,271763
4,B045_PS1_PB00546-01_QC.fcs,/Cells/Singlets-H/Singlets-W/Cleanup/Non-viable,/Cells/Singlets-H/Singlets-W/Cleanup,6587,267721


For analysis, we may want to combine the counts into a single numpy.ndarray object:

In [9]:
counts_arr = numpy.array(
    [counts['Count'].to_list() for counts in counts_list]
)

In [10]:
counts_arr.shape

(8, 63)

We can convert this back to a `DataFrame` if we want to attach names to the dimensions:

In [11]:
counts_df = pandas.DataFrame(
    counts_arr,
    columns = [re.sub(".+/","",x) for x in counts_list[0]['Population']],
    index = counts_info['sample.sampleKitGuid']
)

In [12]:
counts_df[0:10]

,/Cells,Singlets-H,Singlets-W,Cleanup,Non-viable,Viable,Leukocytes,CD19+ Cells,B Cells,CD11c+ B Cells,...,Non-Classical Monocytes,HLA-DR- Cells,Basophils,NK Cells,CD16+ NK Cells,CD16- NK Cells,CD56 High NK Cells,CD56 Low NK Cells,CD34+ Cells,Granulocytes
sample.sampleKitGuid,,,,,,,,,,,,,,,,,,,,,
KT00546,290412,275575,271763,267721,6587,261134,260317,13132,12984,462,...,1392,6876,4112,8414,6775,1639,959,7455,592,198
KT00536,372410,350365,345307,340238,5858,334380,333735,19005,18330,1002,...,4512,6649,2708,29376,26647,2729,2113,27263,504,45
KT00555,338984,319669,314139,309468,5193,304275,303637,13448,12728,1103,...,5547,6396,4206,45353,43961,1392,1304,44049,272,284
KT00554,458791,434690,427267,420110,6352,413758,413217,14942,14153,1315,...,9353,6609,3748,30122,25383,4739,3154,26968,336,47
KT00524,383581,363859,358476,354040,6754,347286,346746,9911,9566,1639,...,4232,3862,2244,79735,74215,5520,3531,76204,426,57
KT00367,337079,317173,312278,308132,7468,300664,299956,20897,20488,1318,...,1669,4809,2098,31351,27040,4311,3964,27387,351,217
KT00505,420222,392814,386699,379405,9225,370180,369767,17116,16587,1270,...,4248,7000,3847,38661,34065,4596,3453,35208,431,120
KT00531,334209,316816,311862,303927,9677,294250,293769,7095,6860,817,...,15278,4692,3381,26265,22827,3438,2102,24163,369,9


### Reading .fcs files

To read .fcs files, we can use the `flowkit` package.

Again, we'll use the previously cached data for this demonstration:

In [13]:
fcs_info = cache_info[cache_info['file.fileType'] == 'FlowCytometry']
fcs_info

,Unnamed: 0,filePath,lastUpdated,sample.id,sample.bridgingControl,sample.sampleKitGuid,sample.visitName,sample.drawDate,sample.daysSinceFirstVisit,file.id,...,file.pool,file.fileType,subject.id,subject.biologicalSex,subject.birthYear,subject.ethnicity,subject.partnerCode,subject.race,subject.subjectGuid,cohort.cohortGuid
5,6,cache/886b5851-f1d7-47fb-a52c-10abec6311d3/B00...,2021-03-19T18:00:54.396Z,cd095a88-11af-4afc-9204-f3210360ac99,False,KT00007,Flu Year 1 Day 7,2019-10-01T00:00:00Z,0,886b5851-f1d7-47fb-a52c-10abec6311d3,...,NaN,FlowCytometry,79ae1d71-8347-4115-a846-2d077cb5077c,Female,1989,non-Hispanic origin,BR,Caucasian,BR1003,BR1


In [14]:
fcs_files = fcs_info['filePath'].to_list()

In [15]:
sample = flowkit.Sample(fcs_files[0])

Once read in, we can perform things like subsampling and transformations within flowkit:

In [16]:
sample.subsample_events(subsample_count = 5000)

In [17]:
xform = flowkit.transforms.LogicleTransform(
    'logicle',
    param_t = 262144,
    param_w = 1.5,
    param_m = 4.5,
    param_a = 0
)
sample.apply_transform(xform)

You may want to export the data to a numpy array or pandas DataFrame for downstream analyses.

The `.get_raw_events()` and `.get_transformed_events()` methods are useful for retrieving data from the `Sample` class.

In [18]:
events = sample.get_transformed_events()
events.shape

(381484, 32)

If you just want the sampled events, use the `subsample` parameter:

In [19]:
events = sample.get_transformed_events(subsample = True)
events.shape

(5000, 32)

A dictionary of channel fluors and feature names can be retrieved using the `.channels` attribute:

In [20]:
channel_dict = sample.channels

In [21]:
channel_dict

{'10': {'PnN': 'BUV615-A', 'PnS': 'CD45RA'},
 '11': {'PnN': 'BUV661-A', 'PnS': 'CD14'},
 '12': {'PnN': 'BUV737-A', 'PnS': 'CD8'},
 '13': {'PnN': 'BUV805-A', 'PnS': 'CD11c'},
 '14': {'PnN': 'BV421-A', 'PnS': 'CD25'},
 '15': {'PnN': 'BV480-A', 'PnS': 'CD4'},
 '16': {'PnN': 'BV510-A', 'PnS': 'Live/Dead'},
 '17': {'PnN': 'BV605-A', 'PnS': 'CD16'},
 '18': {'PnN': 'BV650-A', 'PnS': 'CD123'},
 '19': {'PnN': 'BV711-A', 'PnS': 'CD127'},
 '1': {'PnN': 'FSC-A'},
 '20': {'PnN': 'BV750-P-A', 'PnS': 'IgD'},
 '21': {'PnN': 'BV786-A', 'PnS': 'CD304'},
 '22': {'PnN': 'Alexa Fluor 488-A', 'PnS': 'CD141'},
 '23': {'PnN': 'PerCP-A', 'PnS': 'CD11b'},
 '24': {'PnN': 'BB790-P-A', 'PnS': 'CD19'},
 '25': {'PnN': 'PE-A', 'PnS': 'CD27'},
 '26': {'PnN': 'PE/CF594-A', 'PnS': 'abTCR'},
 '27': {'PnN': 'PE-Cy5-A', 'PnS': 'CD34'},
 '28': {'PnN': 'PE-Cy7-A', 'PnS': 'CD197'},
 '29': {'PnN': 'APC-A', 'PnS': 'CD38'},
 '2': {'PnN': 'FSC-H'},
 '30': {'PnN': 'APC-Alexa 700-A', 'PnS': 'CD56'},
 '31': {'PnN': 'APC-Cy7-A', 'PnS

And basic plotting is well supported:

In [22]:
p = sample.plot_scatter('BV480-A', 'BUV737-A',
                        source = 'xform',
                        subsample = True)
show(p)

In [23]:
p2 = sample.plot_histogram('BUV395-A',
                        source = 'xform',
                        subsample = True)
p2.plot_height = 200
p2.plot_width = 600
show(p2)

### Session Info

In [24]:
import sinfo
sinfo.sinfo(write_req_file = False)

-----
bokeh       2.0.1
flowkit     NA
numpy       1.18.1
pandas      1.0.3
sinfo       0.3.1
-----
IPython             7.13.0
jupyter_client      6.1.3
jupyter_core        4.6.3
jupyterlab          1.2.10
notebook            6.0.3
-----
Python 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) [GCC 7.3.0]
Linux-4.19.0-12-cloud-amd64-x86_64-with-debian-bullseye-sid
4 logical CPU cores, x86_64
-----
Session information updated at 2021-03-29 22:27
